<center><table>
    <tr>
        <th><img src="./INL1.png",align="middle",height="10000"/></th>
        <th><img src="./MIT1.png",align="middle",height="10"\></th>
        <th><img src="./ANL.png",align="middle",height="10"/></th>
    </tr>
</table>
</center>

# <center>Cost Reduction Framework for Nuclear Reactor Power Plants</center>


###  Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from src import prettify, update_high_level_costs, ITC_reduction_factor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', None)

## Section 1 : Reading the Baseline reactor Cost Summary Table 

In [2]:
# A function to read the inital reactor data from excel

def reactor_data_read(reactor_type):
    
    if reactor_type == "Concept A":
        # Reading excel or csv files
        Reactor_data_0 = pd.read_excel('inputs.xlsx', sheet_name = "Concept_A", nrows= 69)
        reactor_power = 1056 * 1000 # kw
        
        
    elif reactor_type == "Concept B":
        # Reading excel or csv files
        Reactor_data_0 = pd.read_excel('inputs.xlsx', sheet_name = "Concept_B", nrows= 69)
        reactor_power = 310.8 * 1000 # kw    
        
    db = pd.DataFrame()
    db = Reactor_data_0[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    Reactor_data = db
    return Reactor_data, reactor_power


# Example
reactor_type = "Concept B"
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]
Reactor_data_pretty = prettify(reactor_data,\
    f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> FOAK Capital Cost Summary - Baseline hypothetical well-executeed project <br><br> ", "no_subsidies")
Reactor_data_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 11,000,000",,,,
12,Site Permits,"$ 1,598,891",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 78,991,565",,,,


## Section - 2 : User Inputs

In [3]:
## ## User-defined Independent Variables (Global Levers)

# reactor type: Concept A or Concept B
reactor_type = "Concept B"

# Which reactor unit: first of a kind : nth of a kind 
n_th = 1

# number of firm orders
num_orders = 13

# land cost
# From the SA report: the cost $22,000 per acre. The land area is 500 acres including recommended buffer
land_cost_per_acre_0 =  22000 # dollars/acre

# start up duration (months) 
startup_0 = 16 

# interest rate :
interest_rate_0 = 0.06

# Design completion
design_completion_0 = 0.8 # 1 means 100%

# Design maturity
Design_Maturity_0 = 1

# #procurement service experience (supply chain experience)
proc_exp_0= 0.5 # 2 means procurement experts. This is ideal. 
   
# #  architecture and engineeringexperience
ae_exp_0 = 0.5
    
# #  Construction service experience
ce_exp_0 = 1


# numb er of projects for full efficiency of procurement, A/E, Construction
N_proc = 3
N_AE  = 4
N_cons =5

# modularity : "stick_built"  or "modularized"
mod_0 = "modularized" 

# cross_site_standardization :
standardization_0 = 0.8 # 0.7 corresponds to 70% standardization for PWRs

# # Determining if the BOP and reactor building (containtment) are non-nuclear or nuclear grade equipment (safety related)
BOP_grade_0 = "non_nuclear"
RB_grade_0 = "nuclear"

# #investment tax credits subsidies
ITC_0 = 0 

#number of reactors claiming ITC
n_ITC = 3 

# factory building cost (associated with accounts 22 and 232.1)
f_22   = 250000000
f_2321 = 150000000


     
global_levers = pd.DataFrame()
global_levers.loc[:, 'Lever'] = ['Design Maturity' ,'Design Completion',\
                                  'Procurement (supply chain) experience ','Architecture & Engineering Experience',\
                                 'Construction service experience',' Land Cost Per Acre (2023 USD)',\
                                'ITC ', ' Interest Rate', 'BOP grade ', 'Reactor Building grade', 'modulariziation',\
                                 'standardization' , "Startup duration (months)"]

global_levers.loc[:, 'User-Input Value'] = [ Design_Maturity_0 ,design_completion_0, proc_exp_0, ae_exp_0, ce_exp_0, 
                                            land_cost_per_acre_0, ITC_0, interest_rate_0, BOP_grade_0  , RB_grade_0 , mod_0,\
                                            standardization_0,  startup_0] 

global_levers.loc[:, 'Lever baseline value (for a hopothetical well-executed project)'] = [ 2, 1, 2, 2, 2, 22000, 0, 0.06 ,\
                                                                                           'nuclear', 'nuclear', "modularized", 0.7 , 16 ]

global_levers.loc[:, 'Range'] = ['0 - 2' ,'0 - 1', '0 - 2', '0 - 2','0 - 2' , '1000 - 100000', '0 - 0.4', '0 - 0.15',\
                                 "nuclear or non-nuclear", "nuclear or non-nuclear",  'stick_built or modularized', '0.7 : 1','3 : 24']



global_levers_changes = global_levers[global_levers['User-Input Value'] != global_levers['Lever baseline value (for a hopothetical well-executed project)']]

slice_ = pd.IndexSlice[global_levers_changes .index, global_levers_changes .columns]
global_levers_styled = (global_levers.style.set_properties(**{'background-color': 'yellow'}, subset=slice_ )\
                            .set_caption("User-Input Global levers <br> (highlighted in yellow if different from the baseline) <br>")\
                        .set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '20px')
    ]
}]))


global_levers_styled.hide()


Lever,User-Input Value,Lever baseline value (for a hopothetical well-executed project),Range
Design Maturity,1,2,0 - 2
Design Completion,0.800000,1,0 - 1
Procurement (supply chain) experience,0.500000,2,0 - 2
Architecture & Engineering Experience,0.500000,2,0 - 2
Construction service experience,1,2,0 - 2
Land Cost Per Acre (2023 USD),22000,22000,1000 - 100000
ITC,0,0,0 - 0.4
Interest Rate,0.060000,0.060000,0 - 0.15
BOP grade,non_nuclear,nuclear,nuclear or non-nuclear
Reactor Building grade,nuclear,nuclear,nuclear or non-nuclear


### The Cost reduction framework: levers and variables impact the costs as shown in the figure (below)

<center><table>
    <tr>
        <th><img src="./framework_diagram.png",align="middle",height="10000"/></th>
    </tr>
</table>
</center>

## Section - 3 : Updating the Cost Summary based on user inputs

### Section - 3-0 : Adding the factory cost to accounts 22 and 232.1

In [4]:
def add_factory_cost(Reactor_data_0, power, f_22, f_2321):
    db = pd.DataFrame()
    db = Reactor_data_0[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    db.loc[db.Account == 22, 'Factory Equipment Cost'] = None # clear old values
    db.loc[db.Account == '232.1', 'Factory Equipment Cost'] = None # clear old values

  
    db.loc[db.Account == 22, 'Factory Equipment Cost'] = ((Reactor_data_0.loc[Reactor_data_0.Account == 22, 'Factory Equipment Cost']) + f_22/num_orders )
    db.loc[db.Account == '232.1', 'Factory Equipment Cost'] = ((Reactor_data_0.loc[Reactor_data_0.Account == '232.1', 'Factory Equipment Cost']) + f_2321/num_orders)


    Reactor_data_fac = update_high_level_costs(db, power)


    Reactor_data_fac_ = pd.DataFrame()
    Reactor_data_fac_ = Reactor_data_fac[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_fac_ 



# Example 

reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

Reactor_data_factory = add_factory_cost(reactor_data , reactor_power, f_22, f_2321)
Reactor_data_factory_pretty = prettify(Reactor_data_factory,\
    f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Factories Cost Included  <br> Displaying Direct Cost only<br><br> ", "no_subsidies")

# list of rows to hide (not affected by this cost change)
list1 = list(range(0, 11))
list2 = list(range(29, 69))
hidden_list1 = list1 + list2

Reactor_data_factory_pretty.hide(subset=hidden_list1, axis=0) # show direct cost only

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 244,678,343","$ 64,350,801","2,899,407 hrs","$ 145,691,462","$ 34,636,081"
212,Reactor Containment Building,"$ 145,108,928","$ 57,583,605","1,605,670 hrs","$ 81,082,169","$ 6,443,153"
213,Turbine Room and Heater Bay,"$ 9,518,418","$ 620,964","102,399 hrs","$ 5,280,509","$ 3,616,945"
211 plus 214 to 219,Othe Structures & Improvements,"$ 90,050,998","$ 6,146,232","1,191,337 hrs","$ 59,328,783","$ 24,575,983"
22,Reactor System,"$ 680,565,566","$ 579,093,613","1,809,704 hrs","$ 97,213,739","$ 4,258,215"
23,Energy Conversion System,"$ 250,382,483","$ 201,341,409","874,096 hrs","$ 46,348,086","$ 2,692,988"
232.1,Electricity Generation Systems,"$ 209,096,582","$ 179,276,621","559,709 hrs","$ 29,819,961",$ 0
233,Ultimate Heat Sink,"$ 41,285,901","$ 22,064,788","314,387 hrs","$ 16,528,125","$ 2,692,988"


### Section 3-1 : The land cost & Taxes

In [5]:
# The cost of the land is 22000$ per acre (500 acres
# The cost is multiplied by the new $/acre divided by the old one
# Accounts 11 and 12 are changed

def add_land_cost(Reactor_data_fac, land_cost_per_acre, power):

    db = pd.DataFrame()
    db = Reactor_data_fac[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

   
    db.loc[db.Account == 11, 'Total Cost (USD)'] = None # clear old values
    db.loc[db.Account == 12, 'Total Cost (USD)'] = None # clear old values
    db.loc[db.Account == 51, 'Total Cost (USD)'] = None # clear old values
    

    db.loc[db.Account == 11, 'Total Cost (USD)'] = (land_cost_per_acre /22000)*(Reactor_data_fac.loc[Reactor_data_fac.Account == 11, 'Total Cost (USD)'].values)
    db.loc[db.Account == 12, 'Total Cost (USD)'] = (land_cost_per_acre /22000)*(Reactor_data_fac.loc[Reactor_data_fac.Account == 12, 'Total Cost (USD)'].values)


    # The taxes scale with increasing the land cost 
    db.loc[db.Account == 51, 'Total Cost (USD)'] = (land_cost_per_acre /22000)*(Reactor_data_fac.loc[Reactor_data_fac.Account == 51, 'Total Cost (USD)'].values)

    db.loc[db.Account == 51, 'Total Cost (USD)']
    Reactor_data_updated_1 = update_high_level_costs(db, power)


    Reactor_data_updated_1_ = pd.DataFrame()
    Reactor_data_updated_1_ = Reactor_data_updated_1[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    return Reactor_data_updated_1_
 

# Example 
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

Reactor_data_factory = add_factory_cost(reactor_data , reactor_power, f_22, f_2321)

Reactor_data_factory_land_taxes = add_land_cost(Reactor_data_factory , land_cost_per_acre_0, reactor_power)
Reactor_data_factory_land_taxes_pretty = prettify(Reactor_data_factory_land_taxes,\
    f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Factories Cost, land cost and taxes Included  <br><br> ", "no_subsidies")

# list of rows to hide (not affected by this cost change)
hidden_list2 = list(range(46, 69))

Reactor_data_factory_land_taxes_pretty.hide(subset=hidden_list2, axis=0) # show direct cost only

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 11,000,000",,,,
12,Site Permits,"$ 1,598,891",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 78,991,565",,,,


### Section 3-2 : Whether the Reactor Building and BOP are nuclear grade equipment

In [6]:
# if the BOP is non-nuclear, the cost reduction factor for account 213 is 0.6
# Also, the cost reduction factor for account 232.1 is 0.6 (for factory and labor but not material)

# If the reactor building is non nuclear, the cost reduction factor for acount 212 is 0.6

def add_BOP_RP_grades(Reactor_data_updated_1, RB_grade_0, BOP_grade_0, power):
    
    RB_grade = RB_grade_0     # Does not change when building more units
    BOP_grade = BOP_grade_0   # Does not change when building more units
    
    db = pd.DataFrame()

    db = Reactor_data_updated_1[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    db.loc[db.Account == 212, 'Site Material Cost'] = None # clear old values
    db.loc[db.Account == 212, 'Site Labor Cost'] = None # clear old values
    db.loc[db.Account ==  212, 'Site Labor Hours'] = None
    db.loc[db.Account ==  212, 'Factory Equipment Cost'] = None

    db.loc[db.Account ==  213, 'Site Material Cost'] = None
    db.loc[db.Account ==  213, 'Site Labor Cost']  = None
    db.loc[db.Account ==  213, 'Site Labor Hours'] = None
    db.loc[db.Account ==  213, 'Factory Equipment Cost'] = None

    db.loc[db.Account ==  '232.1', 'Factory Equipment Cost'] = None
    db.loc[db.Account ==  '232.1', 'Site Labor Cost'] = None
    db.loc[db.Account ==  '232.1', 'Site Labor Hours'] = None


    if RB_grade == "non_nuclear":
        db.loc[db.Account == 212, 'Site Material Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Site Material Cost']).values)
        db.loc[db.Account == 212, 'Site Labor Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Site Labor Cost']).values)
        db.loc[db.Account == 212,'Site Labor Hours'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Site Labor Hours']).values)
        db.loc[db.Account == 212,'Factory Equipment Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Factory Equipment Cost']).values)

    else:
        db.loc[db.Account == 212, 'Site Material Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Site Material Cost']).values)
        db.loc[db.Account == 212, 'Site Labor Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Site Labor Cost']).values)
        db.loc[db.Account == 212,'Site Labor Hours'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Site Labor Hours']).values)
        db.loc[db.Account == 212,'Factory Equipment Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 212, 'Factory Equipment Cost']).values)

    
    if BOP_grade == "non_nuclear":
        db.loc[db.Account == 213, 'Site Material Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Site Material Cost']).values)
        db.loc[db.Account == 213, 'Site Labor Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Site Labor Cost']).values)
        db.loc[db.Account == 213, 'Site Labor Hours'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Site Labor Hours']).values)
        db.loc[db.Account == 213, 'Factory Equipment Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Factory Equipment Cost']).values)

        db.loc[db.Account == '232.1', 'Factory Equipment Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == '232.1', 'Factory Equipment Cost']).values)
        db.loc[db.Account == '232.1', 'Site Labor Hours'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == '232.1', 'Site Labor Hours']).values)
        db.loc[db.Account == '232.1', 'Site Labor Cost'] = 0.6*((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == '232.1', 'Site Labor Cost']).values)


    else:
        db.loc[db.Account == 213, 'Site Material Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Site Material Cost']).values)
        db.loc[db.Account == 213, 'Site Labor Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Site Labor Cost']).values)
        db.loc[db.Account == 213, 'Site Labor Hours'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Site Labor Hours']).values)
        db.loc[db.Account == 213, 'Factory Equipment Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == 213, 'Factory Equipment Cost']).values)

        db.loc[db.Account == '232.1', 'Factory Equipment Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == '232.1', 'Factory Equipment Cost']).values)
        db.loc[db.Account == '232.1', 'Site Labor Hours'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == '232.1', 'Site Labor Hours']).values)
        db.loc[db.Account == '232.1', 'Site Labor Cost'] = ((Reactor_data_updated_1.loc[Reactor_data_updated_1.Account == '232.1', 'Site Labor Cost']).values)


    Reactor_data_updated_2 = update_high_level_costs(db, power)

    Reactor_data_updated_2_ = pd.DataFrame()
    Reactor_data_updated_2_ = Reactor_data_updated_2[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_updated_2_  


# # Example 
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

Reactor_data_factory = add_factory_cost(reactor_data , reactor_power, f_22, f_2321)

Reactor_data_factory_land_taxes = add_land_cost(Reactor_data_factory , land_cost_per_acre_0, reactor_power)
Reactor_data_factory_land_taxes_BOP_RP_grades = add_BOP_RP_grades(Reactor_data_factory_land_taxes  , RB_grade_0, BOP_grade_0, reactor_power)



Reactor_data_factory_land_taxes_BOP_RP_grades_pretty = prettify(Reactor_data_factory_land_taxes_BOP_RP_grades ,\
    f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Factories Cost, land cost, taxes and BOP/RP grades Included  <br> Displaying Direct Cost only<br><br> ", "no_subsidies")



Reactor_data_factory_land_taxes_BOP_RP_grades_pretty.hide(subset=hidden_list1, axis=0) # show direct cost only 


Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 240,870,976","$ 64,102,415","2,858,447 hrs","$ 143,579,258","$ 33,189,303"
212,Reactor Containment Building,"$ 145,108,928","$ 57,583,605","1,605,670 hrs","$ 81,082,169","$ 6,443,153"
213,Turbine Room and Heater Bay,"$ 5,711,051","$ 372,578","61,440 hrs","$ 3,168,305","$ 2,170,167"
211 plus 214 to 219,Othe Structures & Improvements,"$ 90,050,998","$ 6,146,232","1,191,337 hrs","$ 59,328,783","$ 24,575,983"
22,Reactor System,"$ 680,565,566","$ 579,093,613","1,809,704 hrs","$ 97,213,739","$ 4,258,215"
23,Energy Conversion System,"$ 166,743,850","$ 129,630,761","650,212 hrs","$ 34,420,102","$ 2,692,988"
232.1,Electricity Generation Systems,"$ 125,457,949","$ 107,565,972","335,825 hrs","$ 17,891,977",$ 0
233,Ultimate Heat Sink,"$ 41,285,901","$ 22,064,788","314,387 hrs","$ 16,528,125","$ 2,692,988"


### Section 3-3 : Bulk Ordering

In [7]:
#The factory equipment cost of  accounts 22 and 232.1 has to be divided by the number of orders
# The factory equipment cost of account 22 is multiplied by a reuction factor
# The factory equipment cost of account 232.1 is multiplied by a reduction factor


def add_bulk_ordering(Reactor_data_updated_3, num_orders, f_22, f_2321, power):
    db = pd.DataFrame()

    db = Reactor_data_updated_3[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    # esimtated learning rates
    lr22   = 0.1802341659291420
    lr2321 = 0.2607462372040820

    # These reduction factor are calculated as follows
    reduction_factor_22 = 0 # initialization
    reduction_factor_2321 = 0

    for ith_unit in  range(1,num_orders+1):
        reduction_factor_22+=((1 - lr22)**np.log2(ith_unit))/num_orders # for account 22
        reduction_factor_2321+=((1 - lr2321)**np.log2(ith_unit))/num_orders # for account 232.1


        
    for x in [ 22, '232.1']:  
        db.loc[db.Account == x, 'Factory Equipment Cost'] = None # clear old values


    # note that the cost reduction factor impacts the cost of the component not the cost of the factory
    db.loc[db.Account == 22, 'Factory Equipment Cost']       = reduction_factor_22   *  (    ( Reactor_data_updated_3.loc[ Reactor_data_updated_3.Account == 22, 'Factory Equipment Cost']   ) - ( f_22/num_orders) ) + (f_22/num_orders) 
    db.loc[db.Account == '232.1', 'Factory Equipment Cost']  = reduction_factor_2321 *   (  ( Reactor_data_updated_3.loc[ Reactor_data_updated_3.Account == '232.1', 'Factory Equipment Cost']) - ( f_2321/num_orders)) +(f_2321/num_orders) 

    

    Reactor_data_updated_4 = update_high_level_costs(db, power)
    Reactor_data_updated_4_ = pd.DataFrame()
    Reactor_data_updated_4_ = Reactor_data_updated_4[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_updated_4_ 



# # Example 
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

Reactor_data_factory = add_factory_cost(reactor_data , reactor_power, f_22, f_2321)

Reactor_data_factory_land_taxes = add_land_cost(Reactor_data_factory , land_cost_per_acre_0, reactor_power)
Reactor_data_factory_land_taxes_BOP_RP_grades = add_BOP_RP_grades(Reactor_data_factory_land_taxes  , RB_grade_0, BOP_grade_0, reactor_power)
Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder = add_bulk_ordering(Reactor_data_factory_land_taxes_BOP_RP_grades , num_orders, f_22, f_2321, reactor_power)


Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_pretty = prettify(Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder,\
    f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Factories Cost, land cost, taxes, BOP/RP grades , bulk ordering Included  <br> Displaying Direct Cost only<br><br> ", "no_subsidies")
Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_pretty.hide(subset=hidden_list1, axis=0) # show direct cost only

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 240,870,976","$ 64,102,415","2,858,447 hrs","$ 143,579,258","$ 33,189,303"
212,Reactor Containment Building,"$ 145,108,928","$ 57,583,605","1,605,670 hrs","$ 81,082,169","$ 6,443,153"
213,Turbine Room and Heater Bay,"$ 5,711,051","$ 372,578","61,440 hrs","$ 3,168,305","$ 2,170,167"
211 plus 214 to 219,Othe Structures & Improvements,"$ 90,050,998","$ 6,146,232","1,191,337 hrs","$ 59,328,783","$ 24,575,983"
22,Reactor System,"$ 469,343,190","$ 367,871,237","1,809,704 hrs","$ 97,213,739","$ 4,258,215"
23,Energy Conversion System,"$ 118,425,483","$ 81,312,394","650,212 hrs","$ 34,420,102","$ 2,692,988"
232.1,Electricity Generation Systems,"$ 77,139,583","$ 59,247,606","335,825 hrs","$ 17,891,977",$ 0
233,Ultimate Heat Sink,"$ 41,285,901","$ 22,064,788","314,387 hrs","$ 16,528,125","$ 2,692,988"


### Section 3-4 : Reworking and labor productivity

In [8]:
def add_reworking_productivity(Reactor_data_updated_4, reactor_type, n_th, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, power):
    #Reworking = f(AE, CE, design completion)
    
    if n_th == 1:
        design_completion = design_completion_0
        ae_exp = ae_exp_0
        ce_exp = ce_exp_0
        
    elif n_th > 1:
        design_completion = 1   
        ae_exp = min( (ae_exp_0 + (2/N_AE)*(n_th-1) ), 2)
        ce_exp = min( (ce_exp_0 + (2/N_cons)*(n_th-1) ), 2)
    
    # # labor productivity factor = f(construction experience level)
    productivity = 0.145*ce_exp + 0.71        
            

    if reactor_type == "Concept B":
        reworking_factor =  (-0.9*design_completion+ 1.9) * (-0.15*ae_exp+1.3) * (-0.15*ce_exp+1.3) 

    if reactor_type == "Concept A":
        reworking_factor =  (-0.69*design_completion+ 1.69) * (-0.125*ae_exp+1.25) * (-0.125*ce_exp+1.25) 

    db = pd.DataFrame()

    db = Reactor_data_updated_4[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    for x in [212, 213, '211 plus 214 to 219',22, '232.1', 233, 24, 26]:  
        db.loc[db.Account == x, 'Factory Equipment Cost'] = None # clear old values
        db.loc[db.Account == x, 'Site Labor Hours'] = None
        db.loc[db.Account == x, 'Site Labor Cost'] = None
        db.loc[db.Account == x, 'Site Material Cost'] = None

    for x in [212, 213, '211 plus 214 to 219',22, '232.1', 233, 24, 26]: 
        db.loc[db.Account == x, 'Factory Equipment Cost'] =\
            ((( Reactor_data_updated_4.loc[ Reactor_data_updated_4.Account == x, 'Factory Equipment Cost']).values)[0])*reworking_factor  
        db.loc[db.Account == x, 'Site Labor Hours'] =\
            ((( Reactor_data_updated_4.loc[ Reactor_data_updated_4.Account == x, 'Site Labor Hours']).values)[0])*reworking_factor/productivity
        db.loc[db.Account == x, 'Site Labor Cost'] =\
            ((( Reactor_data_updated_4.loc[ Reactor_data_updated_4.Account == x, 'Site Labor Cost']).values)[0])*reworking_factor/productivity 
        db.loc[db.Account == x, 'Site Material Cost'] =\
            ((( Reactor_data_updated_4.loc[ Reactor_data_updated_4.Account == x, 'Site Material Cost']).values)[0])*reworking_factor 


    # update the construction duration

    Reactor_data_updated_5 = update_high_level_costs(db, power)
    Reactor_data_updated_5_ = pd.DataFrame()
    Reactor_data_updated_5_ = Reactor_data_updated_5[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_updated_5_  



# # Example 

reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

Reactor_data_factory = add_factory_cost(reactor_data , reactor_power, f_22, f_2321)

Reactor_data_factory_land_taxes = add_land_cost(Reactor_data_factory , land_cost_per_acre_0, reactor_power)
Reactor_data_factory_land_taxes_BOP_RP_grades = add_BOP_RP_grades(Reactor_data_factory_land_taxes  , RB_grade_0, BOP_grade_0, reactor_power)
Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder = add_bulk_ordering(Reactor_data_factory_land_taxes_BOP_RP_grades , num_orders, f_22, f_2321, reactor_power)
Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_rework_productivity =\
    add_reworking_productivity(Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder , reactor_type, n_th , design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, reactor_power)


Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_rework_productivity_pretty = prettify(Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_rework_productivity,\
        f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor Capital Cost Summary <br> Factories Cost, land cost, taxes, BOP/RP grades , bulk ordering, reworking and productivity Included  <br> Displaying Direct Cost only<br><br> ", "no_subsidies")
Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_rework_productivity_pretty.hide(subset=hidden_list1, axis=0) # show direct cost only 

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 440,882,959","$ 106,559,047","5,557,506 hrs","$ 279,152,503","$ 55,171,408"
212,Reactor Containment Building,"$ 264,076,459","$ 95,722,667","3,121,808 hrs","$ 157,643,178","$ 10,710,615"
213,Turbine Room and Heater Bay,"$ 10,386,814","$ 619,346","119,453 hrs","$ 6,159,946","$ 3,607,522"
211 plus 214 to 219,Othe Structures & Improvements,"$ 166,419,685","$ 10,217,034","2,316,244 hrs","$ 115,349,380","$ 40,853,271"
22,Reactor System,"$ 807,606,906","$ 611,521,554","3,518,498 hrs","$ 189,006,816","$ 7,078,536"
23,Energy Conversion System,"$ 206,565,176","$ 135,167,625","1,264,169 hrs","$ 66,920,930","$ 4,476,621"
232.1,Electricity Generation Systems,"$ 133,275,069","$ 98,488,776","652,925 hrs","$ 34,786,293",$ 0
233,Ultimate Heat Sink,"$ 73,290,107","$ 36,678,849","611,244 hrs","$ 32,134,637","$ 4,476,621"


#### Combine the previous functions in one function

In [9]:
# Combine the previous functions in one function

def update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons):
    reactor_data = (reactor_data_read(reactor_type ))[0]
    reactor_power = (reactor_data_read(reactor_type ))[1]
    Reactor_data_factory = add_factory_cost(reactor_data , reactor_power, f_22, f_2321)
    Reactor_data_factory_land_taxes = add_land_cost(Reactor_data_factory , land_cost_per_acre_0, reactor_power)
    Reactor_data_factory_land_taxes_BOP_RP_grades = add_BOP_RP_grades(Reactor_data_factory_land_taxes  , RB_grade_0, BOP_grade_0, reactor_power)
    Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder = add_bulk_ordering(Reactor_data_factory_land_taxes_BOP_RP_grades , num_orders, f_22, f_2321, reactor_power)
    
    Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_rework_productivity =\
    add_reworking_productivity(Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder , reactor_type, n_th , design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, reactor_power)
    
    return Reactor_data_factory_land_taxes_BOP_RP_grades_bulkOrder_rework_productivity

# example 

reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)
direct_cost_updated_pretty = prettify(direct_cost_updated,\
        f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Direct Cost Updated <br><br> ", "no_subsidies")
direct_cost_updated_pretty.hide(subset=hidden_list1, axis=0) # show direct cost only 

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 440,882,959","$ 106,559,047","5,557,506 hrs","$ 279,152,503","$ 55,171,408"
212,Reactor Containment Building,"$ 264,076,459","$ 95,722,667","3,121,808 hrs","$ 157,643,178","$ 10,710,615"
213,Turbine Room and Heater Bay,"$ 10,386,814","$ 619,346","119,453 hrs","$ 6,159,946","$ 3,607,522"
211 plus 214 to 219,Othe Structures & Improvements,"$ 166,419,685","$ 10,217,034","2,316,244 hrs","$ 115,349,380","$ 40,853,271"
22,Reactor System,"$ 807,606,906","$ 611,521,554","3,518,498 hrs","$ 189,006,816","$ 7,078,536"
23,Energy Conversion System,"$ 206,565,176","$ 135,167,625","1,264,169 hrs","$ 66,920,930","$ 4,476,621"
232.1,Electricity Generation Systems,"$ 133,275,069","$ 98,488,776","652,925 hrs","$ 34,786,293",$ 0
233,Ultimate Heat Sink,"$ 73,290,107","$ 36,678,849","611,244 hrs","$ 32,134,637","$ 4,476,621"


### Section 3-5 :Update construction duration from labor hours

In [10]:

def update_cons_dur(Reactor_data_0, db, mod_0):

    #sum of labor hours for Account 20 in the initial estimation (well exectued scenario)
    sum_old_lab_hrs = (Reactor_data_0.loc[Reactor_data_0.Account == 21, 'Site Labor Hours']).values +\
    (Reactor_data_0.loc[Reactor_data_0.Account == 22, 'Site Labor Hours']).values +\
    (Reactor_data_0.loc[Reactor_data_0.Account == 23, 'Site Labor Hours']).values+\
    (Reactor_data_0.loc[Reactor_data_0.Account == 24, 'Site Labor Hours']).values+\
    (Reactor_data_0.loc[Reactor_data_0.Account == 26, 'Site Labor Hours']).values



    # #sum of labor hours for Account 20 in the new estimation 
    sum_new_lab_hrs = (db.loc[db.Account == 21, 'Site Labor Hours']).values +\
    (db.loc[db.Account == 22, 'Site Labor Hours']).values +\
    (db.loc[db.Account == 23, 'Site Labor Hours']).values+\
    (db.loc[db.Account == 24, 'Site Labor Hours']).values+\
    (db.loc[db.Account == 26, 'Site Labor Hours']).values



    # # # change in labor hours for account 20
    labor_hour_ratio = (sum_new_lab_hrs)/sum_old_lab_hrs   # note that this number can be positive or negative
    labor_hour_ratio 

    # 	From the literature   we know that if labor hours changed from 3.8M hours to 20.5M hours (5.4 times), the construction duration changes from 33.2 months to 74.3 months (2.2 times).
    # 	We also know that if the labor hours multiplier =1, the cons duration multiplier should be 1.
    # 	Using these two points: 

    # construction duration multiplier = 0.3 * the labor hours multiplier+0.7



    #     # modularity (applied on civil construction only) "stick_built"  or "modularized"
    mod = mod_0

    # Modularization effect on the construction duration
    if mod == "stick_built":
        mod_factor = 0.8
    elif mod == "modularized":
        mod_factor = 1
        

    # # This is a hypothetical well-executed project taking 64 months (TIMCAT simulation)
    if reactor_type ==  "Concept B": 
        baseline_construction_duration = 64/mod_factor # months
    elif reactor_type == "Concept A":  
        baseline_construction_duration = 100/mod_factor # months

    actual_construction_duration = baseline_construction_duration*(0.3*labor_hour_ratio+0.7)
    return actual_construction_duration 

# example

direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)
act_con_duration = update_cons_dur(reactor_data, direct_cost_updated , mod_0)
# print(f"The construction duration of the {reactor_type} reactor, without accounting for supply chain dealys, is { np.round( act_con_duration[0]/12 , 1) } years")

### Section 3-6 Learning by doing effect on the cost

In [11]:

def learning_effect(Reactor_data_updated_5, n_th, standardization_0, power):

    if n_th  == 1:
        standardization = 0.7
    elif n_th >1:
        standardization = standardization_0    
        
    # # Creating the table for the learning rates
    # #These rates are from KS-TIMCAT results.
    # # The learning rates are multiplied by the standardization divded by 0.7 (since the standatization of PWRs was 0.7)
    fitted_LR = pd.DataFrame()

    fitted_LR.loc[:, 'Account'] = Reactor_data_updated_5.loc[:, 'Account']
    fitted_LR.loc[:, 'Title'] = Reactor_data_updated_5.loc[:, 'Title']
    fitted_LR = fitted_LR.loc[fitted_LR['Account'].isin([212, 213, '211 plus 214 to 219', 22, '232.1', 233, 24, 26])]

    fitted_LR['Mat LR'] = np.array([0.099588665391, 0.099588665391, 0.099588665391, 0.080817992281, 0.0000000, 0.099588665391,\
                                    0.099588665391,0.099588665391])*standardization/0.7

    fitted_LR['Lab LR'] =     np.array([0.180678729399, 0.180678729399, 0.180678729399,0.146555539499, 0.137148574884,\
                                        0.180678729399, 0.180678729399,0.180678729399])*standardization/0.7


    db = pd.DataFrame()

    db = Reactor_data_updated_5[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    for x in [212, 213, '211 plus 214 to 219', 22, '232.1', 233, 24, 26]:  
        db.loc[db.Account == x, 'Site Labor Hours'] = None
        db.loc[db.Account == x, 'Site Labor Cost'] = None
        db.loc[db.Account == x, 'Site Material Cost'] = None


    # # # # Bulk order reduction	
    for x in [212, 213, '211 plus 214 to 219', 22, '232.1', 233, 24, 26]:
        mat_cost_reduction_multiplier = (1 - (fitted_LR.loc[fitted_LR.Account == x, 'Mat LR'].values[0]))**np.log2(n_th)
        lab_cost_reduction_multiplier = (1 - (fitted_LR.loc[fitted_LR.Account == x, 'Lab LR'].values[0]))**np.log2(n_th)
        
        db.loc[db.Account == x, 'Site Material Cost'] =\
            (( Reactor_data_updated_5.loc[ Reactor_data_updated_5.Account == x, 'Site Material Cost']))* mat_cost_reduction_multiplier

        db.loc[db.Account == x, 'Site Labor Hours'] =\
            (( Reactor_data_updated_5.loc[ Reactor_data_updated_5.Account == x, 'Site Labor Hours']))* lab_cost_reduction_multiplier
    
        db.loc[db.Account == x, 'Site Labor Cost'] =\
            (( Reactor_data_updated_5.loc[ Reactor_data_updated_5.Account == x, 'Site Labor Cost']))* lab_cost_reduction_multiplier


    Reactor_data_updated_6 = update_high_level_costs(db, power)
    Reactor_data_updated_6_ = pd.DataFrame()
    Reactor_data_updated_6_ = Reactor_data_updated_6[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()
    
    return Reactor_data_updated_6_  


# # Example 
reactor_power = (reactor_data_read(reactor_type ))[1]
direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)
direct_cost_updated_plus_learning = learning_effect(direct_cost_updated , n_th, standardization_0, reactor_power)

direct_cost_updated_plus_learning_pretty = prettify(direct_cost_updated_plus_learning,\
        f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Direct Cost Updated plus learning effect <br> Displaying Direct Cost only<br><br> ", "no_subsidies")
direct_cost_updated_plus_learning_pretty.hide(subset=hidden_list1, axis=0) # show direct cost only  

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 440,882,959","$ 106,559,047","5,557,506 hrs","$ 279,152,503","$ 55,171,408"
212,Reactor Containment Building,"$ 264,076,459","$ 95,722,667","3,121,808 hrs","$ 157,643,178","$ 10,710,615"
213,Turbine Room and Heater Bay,"$ 10,386,814","$ 619,346","119,453 hrs","$ 6,159,946","$ 3,607,522"
211 plus 214 to 219,Othe Structures & Improvements,"$ 166,419,685","$ 10,217,034","2,316,244 hrs","$ 115,349,380","$ 40,853,271"
22,Reactor System,"$ 807,606,906","$ 611,521,554","3,518,498 hrs","$ 189,006,816","$ 7,078,536"
23,Energy Conversion System,"$ 206,565,176","$ 135,167,625","1,264,169 hrs","$ 66,920,930","$ 4,476,621"
232.1,Electricity Generation Systems,"$ 133,275,069","$ 98,488,776","652,925 hrs","$ 34,786,293",$ 0
233,Ultimate Heat Sink,"$ 73,290,107","$ 36,678,849","611,244 hrs","$ 32,134,637","$ 4,476,621"


### Section 3-7 supply chain delays

In [12]:
def act_cons_duration_plus_delay(reactor_type, n_th, Design_Maturity_0, proc_exp_0, N_proc, cons_duration_no_delay):
    
    if n_th == 1:
        Design_Maturity = Design_Maturity_0
        proc_exp = proc_exp_0
    
    elif n_th>1:
        Design_Maturity = 2
        proc_exp = min( (proc_exp_0 + (2/N_proc)*(n_th-1) ), 2)
             
    # For the accounts 21, 22, 23, 24, 25, 26, 
    # The delays are D_21,D_22,D_23,D_24,D_25,D_26 
    # The tasks lengths (in months) are B_21,B_22,B_23,B_24,B_25,B_26

    # task length ratio between SFR and HTGR is 100/65
    # We use this ratio to convert SFR to HTGT task lengths

    if reactor_type == "Concept B":
        task_length_multiplier = 1
        ref_construction_duration = 64
    elif reactor_type == "Concept A": 
        task_length_multiplier = 100/64
        ref_construction_duration = 100
        
    B_21 =  42.1 * task_length_multiplier # months 
    B_22 = 60.2 * task_length_multiplier
    B_23 = 14.8 * task_length_multiplier
    B_24 = 3.6* task_length_multiplier
    B_25 = 10.1* task_length_multiplier
    B_26 = 43.9* task_length_multiplier


    # The delays are 
    D_21 =  - 6 * Design_Maturity  - 3*proc_exp + 18
    D_22 =  - 6 * Design_Maturity  - 3*proc_exp + 18
    D_23 =   - 6 * Design_Maturity  - 3*proc_exp + 18
    D_24 =   - 6 * Design_Maturity  - 3*proc_exp + 18
    D_25 =   - 6 * Design_Maturity  - 3*proc_exp + 18
    D_26 =   - 6 * Design_Maturity  - 3*proc_exp + 18


    # The tasks completion times (in months) are T_21,T_22,T_23,T_24,T_25,T_26
    T_21 = B_21 + D_21 
    T_22 = 0.09*(B_21+D_21)  +B_22+D_22
    T_23 = 0.24*(B_21+D_21)  +B_23+D_23
    T_24 = 0.24*(B_21+D_21) + 0.34*(B_23+D_23)  +B_24+D_24
    T_25 = 0.18*(B_21+D_21)  +B_25+D_25
    T_26 = 0.21*(B_21+D_21)   +B_26+D_26
    T_end = max(T_21, T_22, T_23, T_24, T_25, T_26)


    supply_chain_delay = max( T_end - ref_construction_duration, 0)
    actual_construction_duration_plus_delay = cons_duration_no_delay + supply_chain_delay
    return actual_construction_duration_plus_delay





# example
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]

direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)
act_con_duration = update_cons_dur(reactor_data, direct_cost_updated , mod_0)

cons_duration_plus_delay = act_cons_duration_plus_delay(reactor_type, n_th, Design_Maturity_0, proc_exp_0, N_proc, act_con_duration)
# print(f"The construction duration of the {reactor_type} reactor, accounting for supply chain dealys, is { np.round( cons_duration_plus_delay[0]/12 , 1) } years")

### Section 3-8 Learning by doing effect on the construction duration

In [13]:
def duration_learning_effect(n_th, standardization_0, actual_construction_duration_plus_delay):
    
    if n_th  == 1:
        standardization = 0.7
    elif n_th >1:
        standardization = standardization_0  
        
    # now the effect of learning on the consturction duration
    fitted_LR_duration = 0.15*standardization/0.7
    duration_multiplier = (1 -fitted_LR_duration)**np.log2(n_th)
    final_construction_duration = duration_multiplier *actual_construction_duration_plus_delay
    return final_construction_duration 

# example
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]
direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)
act_con_duration = update_cons_dur(reactor_data, direct_cost_updated , mod_0)

cons_duration_plus_delay = act_cons_duration_plus_delay(reactor_type, n_th, Design_Maturity_0, proc_exp_0, N_proc, act_con_duration)
final_con_duration = duration_learning_effect(n_th, standardization_0, cons_duration_plus_delay)
# print(f"The final construction duration of the {reactor_type} reactor is { np.round( final_con_duration[0]/12 , 1) } years")

### Section 3-9 Calculate the Indirect Cost and the standardization impact

In [14]:
def update_indirect_cost(n_th, standardization_0, Reactor_data_updated_6, final_construction_duration, power ):
    
    if n_th  == 1:
        standardization = 0.7
    elif n_th >1:
        standardization = standardization_0     
    #   I use here the indirect cost correlations prepared by Jia

    # When the standardization is  100%, the engineering service accound (Acct 35) is zero, 
    # When the standardization is  70%, the engineering service accound (Acct 35) does not change
    # The account 35 multiplier is


    factor_35 = -3.33 * standardization + 3.331


    db = pd.DataFrame()

    db = Reactor_data_updated_6[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    for x in [31, 32, 33, 34, 35]:  
        db.loc[db.Account == x, 'Total Cost (USD)'] = None # clear old values


    # # total direct mat cost, labor cost, labor hours
    sum_new_mat_cost  = 0 # initilization
    sum_new_lab_cost  = 0 # initilization
    sum_new_lab_hrs  = 0 # initilization

    for x in [21, 22, 23, 24, 26]:
        sum_new_mat_cost +=     (db.loc[db.Account == x, 'Site Material Cost']).values
        sum_new_lab_cost +=     (db.loc[db.Account == x, 'Site Labor Cost']).values
        sum_new_lab_hrs  +=     (db.loc[db.Account == x, 'Site Labor Hours']).values


    # The new indirect costs   
    db.loc[db.Account == 31, 'Total Cost (USD)'] =  (sum_new_mat_cost*0.785* sum_new_lab_hrs/final_construction_duration/160/1058)\
    + sum_new_lab_cost *0.36

    db.loc[db.Account == 32, 'Total Cost (USD)'] =  sum_new_lab_cost *0.36*3.661* final_construction_duration/72

    db.loc[db.Account == 33, 'Total Cost (USD)'] =  0.042 * (db.loc[db.Account == 32, 'Total Cost (USD)'].values[0] )

    db.loc[db.Account == 34, 'Total Cost (USD)'] =  0.0035 * (db.loc[db.Account == 32, 'Total Cost (USD)'].values[0] )

    db.loc[db.Account == 35, 'Total Cost (USD)'] = ( 0.27 * (db.loc[db.Account == 32, 'Total Cost (USD)'].values[0] ))*factor_35 


    Reactor_data_updated_7 = update_high_level_costs(db, power)
    Reactor_data_updated_7_ = pd.DataFrame()
    Reactor_data_updated_7_ = Reactor_data_updated_7[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_updated_7_  




# # Example 
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]
direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)

act_con_duration = update_cons_dur(reactor_data, direct_cost_updated , mod_0)
cons_duration_plus_delay = act_cons_duration_plus_delay(reactor_type, n_th, Design_Maturity_0, proc_exp_0, N_proc, act_con_duration)
final_con_duration = duration_learning_effect(n_th, standardization_0, cons_duration_plus_delay)

direct_cost_updated_plus_learning = learning_effect(direct_cost_updated , n_th, standardization_0, reactor_power)
direct_cost_updated_plus_learning_with_indirect_cost = update_indirect_cost(n_th, standardization_0, direct_cost_updated_plus_learning, final_con_duration, reactor_power  )

direct_cost_updated_plus_learning_with_indirect_cost_pretty = prettify(direct_cost_updated_plus_learning_with_indirect_cost,\
        f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Base Cost updated (Direct and Indirect costs) <br><br> ", "no_subsidies")

list3 = list(range(38, 69))
hidden_list3 = list1 + list3
direct_cost_updated_plus_learning_with_indirect_cost_pretty.hide(subset=hidden_list3, axis=0) # show direct and indirect cost only 

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 440,882,959","$ 106,559,047","5,557,506 hrs","$ 279,152,503","$ 55,171,408"
212,Reactor Containment Building,"$ 264,076,459","$ 95,722,667","3,121,808 hrs","$ 157,643,178","$ 10,710,615"
213,Turbine Room and Heater Bay,"$ 10,386,814","$ 619,346","119,453 hrs","$ 6,159,946","$ 3,607,522"
211 plus 214 to 219,Othe Structures & Improvements,"$ 166,419,685","$ 10,217,034","2,316,244 hrs","$ 115,349,380","$ 40,853,271"
22,Reactor System,"$ 807,606,906","$ 611,521,554","3,518,498 hrs","$ 189,006,816","$ 7,078,536"
23,Energy Conversion System,"$ 206,565,176","$ 135,167,625","1,264,169 hrs","$ 66,920,930","$ 4,476,621"
232.1,Electricity Generation Systems,"$ 133,275,069","$ 98,488,776","652,925 hrs","$ 34,786,293",$ 0
233,Ultimate Heat Sink,"$ 73,290,107","$ 36,678,849","611,244 hrs","$ 32,134,637","$ 4,476,621"


#### Combine the previous functions in one function

In [15]:
def calculate_base_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, mod_0, Design_Maturity_0, proc_exp_0, N_proc, standardization_0):
    reactor_data = (reactor_data_read(reactor_type ))[0]
    reactor_power = (reactor_data_read(reactor_type ))[1]
    direct_cost_updated = update_direct_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons)

    act_con_duration = update_cons_dur(reactor_data, direct_cost_updated , mod_0)
    cons_duration_plus_delay = act_cons_duration_plus_delay(reactor_type, n_th, Design_Maturity_0, proc_exp_0, N_proc, act_con_duration)
    final_con_duration = duration_learning_effect(n_th, standardization_0, cons_duration_plus_delay)

    direct_cost_updated_plus_learning = learning_effect(direct_cost_updated , n_th, standardization_0, reactor_power)
    direct_cost_updated_plus_learning_with_indirect_cost = update_indirect_cost(n_th, standardization_0, direct_cost_updated_plus_learning, final_con_duration, reactor_power  )
    return direct_cost_updated_plus_learning_with_indirect_cost, final_con_duration 

### Section 3-10 :  Insurance

In [16]:
def insurance_cost_update(Reactor_data_0, Reactor_data_updated_7, power):
    # insurance increases linearly when increaing the sum of the 20s and 30s account
    db = pd.DataFrame()

    db = Reactor_data_updated_7[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    db0 = Reactor_data_0
    db.loc[db.Account == 52, 'Total Cost (USD)'] = None # clear old values

    change_in_insuance_cost = (db.loc[db.Title =='20s - Subtotal', 'Total Cost (USD)'].values\
                            + db.loc[db.Title =='30s - Subtotal', 'Total Cost (USD)'].values)/ (db0.loc[db0.Title =='20s - Subtotal', 'Total Cost (USD)'].values\
                            + db0.loc[db0.Title =='30s - Subtotal', 'Total Cost (USD)'].values)

    db.loc[db.Account == 52, 'Total Cost (USD)'] =  (change_in_insuance_cost[0])* (Reactor_data_updated_7.loc[db.Account == 52, 'Total Cost (USD)'])

    Reactor_data_updated_8 = update_high_level_costs(db, power)

    Reactor_data_updated_8_ = pd.DataFrame()
    Reactor_data_updated_8_ = Reactor_data_updated_8[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_updated_8_  

# Example
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]
tot_base_cost = (calculate_base_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, mod_0, Design_Maturity_0, proc_exp_0, N_proc, standardization_0))[0]
tot_base_cost_wz_insurance = insurance_cost_update(reactor_data, tot_base_cost , reactor_power)
tot_base_cost_wz_insurance_pretty = prettify(tot_base_cost_wz_insurance,\
            f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Base Cost updated (Direct and Indirect costs) plus insurance <br><br> ", "no_subsidies")


list4 = list(range(46, 69))
hidden_list4 = list1 + list4
tot_base_cost_wz_insurance_pretty.hide(subset=hidden_list4, axis=0)  

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
,,,,,,
20,Capitalized Direct Costs,,,,,
21,Structures & Improvements,"$ 440,882,959","$ 106,559,047","5,557,506 hrs","$ 279,152,503","$ 55,171,408"
212,Reactor Containment Building,"$ 264,076,459","$ 95,722,667","3,121,808 hrs","$ 157,643,178","$ 10,710,615"
213,Turbine Room and Heater Bay,"$ 10,386,814","$ 619,346","119,453 hrs","$ 6,159,946","$ 3,607,522"
211 plus 214 to 219,Othe Structures & Improvements,"$ 166,419,685","$ 10,217,034","2,316,244 hrs","$ 115,349,380","$ 40,853,271"
22,Reactor System,"$ 807,606,906","$ 611,521,554","3,518,498 hrs","$ 189,006,816","$ 7,078,536"
23,Energy Conversion System,"$ 206,565,176","$ 135,167,625","1,264,169 hrs","$ 66,920,930","$ 4,476,621"
232.1,Electricity Generation Systems,"$ 133,275,069","$ 98,488,776","652,925 hrs","$ 34,786,293",$ 0
233,Ultimate Heat Sink,"$ 73,290,107","$ 36,678,849","611,244 hrs","$ 32,134,637","$ 4,476,621"


### Section 3-11 :  Interest

In [17]:
def update_interest_cost(Reactor_data_updated_8, final_construction_duration, interest_rate, startup_0, n_th, power):
    # Read the ref spending curve
    sp = pd.read_excel('inputs.xlsx', sheet_name = "Ref Spending Curve", nrows= 104, usecols='A : D')
    Months = sp['Month'].tolist()
    CDFs  = sp['CDF'].tolist()

    annual_periods = np.linspace(12, 12*int(final_construction_duration/12),int(final_construction_duration/12))
    if max(annual_periods) < int(final_construction_duration)-1:
        annual_periods_1= np.append( annual_periods, (final_construction_duration)-1)
    else:
        annual_periods_1 = annual_periods
    annual_cum_spend = []
    for period in annual_periods_1:
        new_period = 103*period/int(final_construction_duration)
        annual_cum_spend.append(np.interp(new_period, Months, CDFs))
        
    annual_cum_spend1 = np.append(annual_cum_spend[0], np.diff(annual_cum_spend ))
    tot_overnight_cost =   (Reactor_data_updated_8.loc[Reactor_data_updated_8.Title == 'Total Overnight Cost (Accounts 10 to 50)' , 'Total Cost (USD)']).values[0]


    annual_loan_add =  annual_cum_spend1 *tot_overnight_cost

    interest_exp = ((1+interest_rate)**((final_construction_duration -annual_periods_1)/12)) * annual_loan_add - annual_loan_add

    tot_int_exp_construction = sum(interest_exp )
    
    if n_th == 1:
        startup = startup_0
    elif n_th >1:
        startup = max( 7 , startup_0*(1-0.3)**np.log2(n_th)  )

    int_exp_startup = (tot_int_exp_construction + tot_overnight_cost)*((1+interest_rate)**(startup/12))-(tot_int_exp_construction + tot_overnight_cost)
    
    db = pd.DataFrame()

    db = Reactor_data_updated_8[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    db.loc[db.Account == 62, 'Total Cost (USD)'] = None # clear old values

    (db.loc[db.Account == 62, 'Total Cost (USD)']) = int_exp_startup +tot_int_exp_construction 


    Reactor_data_updated_9 = update_high_level_costs(db, power)

    Reactor_data_updated_9_ = pd.DataFrame()
    Reactor_data_updated_9_ = Reactor_data_updated_9[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    tot_cap_investment = Reactor_data_updated_9.loc[Reactor_data_updated_9.Title =='Total Capital Investment Cost (All Accounts)', 'Total Cost (USD)'].values
    
    return Reactor_data_updated_9_, tot_overnight_cost, tot_cap_investment



# Example
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]
tot_base_cost_results = (calculate_base_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, mod_0, Design_Maturity_0, proc_exp_0, N_proc, standardization_0))
tot_base_cost = tot_base_cost_results[0]
final_construction_duration = tot_base_cost_results[1]

tot_base_cost_wz_insurance = insurance_cost_update(reactor_data, tot_base_cost , reactor_power)
tot_base_cost_wz_insurance_interest = (update_interest_cost(tot_base_cost_wz_insurance , final_construction_duration, interest_rate_0, startup_0, n_th, reactor_power))[0]



tot_base_cost_wz_insurance_interest_pretty = prettify(tot_base_cost_wz_insurance_interest,\
            f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary - Base Cost updated (Direct and Indirect costs) plus insurance and interest <br><br> ", "no_subsidies")

tot_base_cost_wz_insurance_interest_pretty


Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 11,000,000",,,,
12,Site Permits,"$ 1,598,891",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 78,991,565",,,,


### Section 3 - 12 :  ITC Subsidies

In [18]:
def update_itc(Reactor_data_updated_9, tot_overnight_cost, tot_cap_investment, n_th, ITC_0, n_ITC, reactor_power):
    
    if n_th <= n_ITC:
        ITC = ITC_0
    else:
        ITC =0
    
    db1 = pd.DataFrame()
    db1 = Reactor_data_updated_9[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()

    ITC_cost_reduction_factor = ITC_reduction_factor(ITC)

    ITC_reduced_OCC =tot_overnight_cost*ITC_cost_reduction_factor
    OCC_cost_reduction_due_to_TCI =  tot_overnight_cost -  ITC_reduced_OCC 

    db1.loc[db1.Title== 'Total Overnight Cost - ITC reduced', 'Total Cost (USD)'] = None # clear old values
    db1.loc[db1.Title== 'Total Overnight Cost -ITC reduced (US$/kWe)', 'Total Cost (USD)'] = None # clear old values
    db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced', 'Total Cost (USD)'] = None # clear old values
    db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced (US$/kWe)', 'Total Cost (USD)'] = None

    db1.loc[db1.Title== 'Total Overnight Cost - ITC reduced', 'Total Cost (USD)'] = ITC_reduced_OCC
    db1.loc[db1.Title== 'Total Overnight Cost -ITC reduced (US$/kWe)', 'Total Cost (USD)'] = ITC_reduced_OCC/reactor_power
    db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced', 'Total Cost (USD)'] = tot_cap_investment - OCC_cost_reduction_due_to_TCI 
    
    #Net capital investment_per_kw
    levelized_NCI = (db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced', 'Total Cost (USD)'].values[0])/reactor_power
    db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced (US$/kWe)', 'Total Cost (USD)'] = levelized_NCI


    Reactor_data_updated_10 = update_high_level_costs(db1, reactor_power)

    Reactor_data_updated_10_ = pd.DataFrame()
    Reactor_data_updated_10_ = Reactor_data_updated_10[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                    'Site Material Cost']].copy()
    
    return Reactor_data_updated_10_ , ITC_reduced_OCC/reactor_power, levelized_NCI 





# Example
reactor_data = (reactor_data_read(reactor_type ))[0]
reactor_power = (reactor_data_read(reactor_type ))[1]
tot_base_cost_results = (calculate_base_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, mod_0, Design_Maturity_0, proc_exp_0, N_proc, standardization_0))
tot_base_cost = tot_base_cost_results[0]
final_construction_duration = tot_base_cost_results[1]

tot_base_cost_wz_insurance = insurance_cost_update(reactor_data, tot_base_cost , reactor_power)
tot_base_cost_wz_insurance_interest_results = (update_interest_cost(tot_base_cost_wz_insurance , final_construction_duration, interest_rate_0, startup_0, n_th, reactor_power))
tot_base_cost_wz_insurance_interest= tot_base_cost_wz_insurance_interest_results[0]
tot_overnight_cost = tot_base_cost_wz_insurance_interest_results[1]
tot_cap_investment= tot_base_cost_wz_insurance_interest_results[2]



Final_Result = update_itc(tot_base_cost_wz_insurance_interest, tot_overnight_cost, tot_cap_investment, n_th, ITC_0, n_ITC, reactor_power)
Final_Result_COA = Final_Result[0]

Final_Result_pretty = prettify(Final_Result_COA,\
            f" The {reactor_type} {np.round(reactor_power/1000,1)} MWe Reactor <br> Capital Cost Summary  <br><br> ", "subsidies")

Final_Result_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 11,000,000",,,,
12,Site Permits,"$ 1,598,891",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 78,991,565",,,,


#### A Python function to combine all the previous ones

In [19]:
def calculate_final_result(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0,\
    num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, mod_0, Design_Maturity_0, proc_exp_0, N_proc, standardization_0, interest_rate_0, startup_0): 

    reactor_data = (reactor_data_read(reactor_type ))[0]
    reactor_power = (reactor_data_read(reactor_type ))[1]
    tot_base_cost_results = calculate_base_cost(reactor_type, n_th, f_22, f_2321, land_cost_per_acre_0, RB_grade_0, BOP_grade_0, num_orders, design_completion_0, ae_exp_0, N_AE, ce_exp_0, N_cons, mod_0, Design_Maturity_0, proc_exp_0, N_proc, standardization_0)
    tot_base_cost = tot_base_cost_results[0]
    final_construction_duration = tot_base_cost_results[1]

    tot_base_cost_wz_insurance = insurance_cost_update(reactor_data, tot_base_cost , reactor_power)
    tot_base_cost_wz_insurance_interest_results = (update_interest_cost(tot_base_cost_wz_insurance , final_construction_duration, interest_rate_0, startup_0, n_th, reactor_power))
    tot_base_cost_wz_insurance_interest= tot_base_cost_wz_insurance_interest_results[0]
    tot_overnight_cost = tot_base_cost_wz_insurance_interest_results[1]
    tot_cap_investment= tot_base_cost_wz_insurance_interest_results[2]

    Final_Result = update_itc(tot_base_cost_wz_insurance_interest, tot_overnight_cost, tot_cap_investment, n_th, ITC_0, n_ITC, reactor_power)
    Final_Result_COA = Final_Result[0]
    levelized_net_OCC = Final_Result[1]
    levelized_NCI = Final_Result[2]
    
    return Final_Result_COA, levelized_net_OCC, levelized_NCI, final_construction_duration 